# Pruning BERT/SQuAD Model
This notebook shows how to reduce the size of a model by pruning its parameters. It assumes 
that a trained model already exists in the ```Models``` directory. Please refer to the notebook [Question Answering (BERT/SQuAD)](BertSquad.ipynb) for more info about training and using a BERT/SQuAD model.

If you want to prune a Low-Rank model, you can use [this](BertSquad-Reduce.ipynb) notebook
to reduce the number of parameters in ```BERT/SQuAD```.

## Load and evaluate the original pretrained model

In [1]:
from fireball import Model, myPrint
from fireball.datasets.squad import SquadDSet
import time, os

gpus = "0,1,2,3"

testDs = SquadDSet.makeDatasets("Test", batchSize=128, version=1 )

# orgFileName = "Models/BertSquad.fbm"    # Original model
# orgFileName = "Models/BertSquadR.fbm"   # Reduced
orgFileName = "Models/BertSquadRR.fbm"    # Reduced - Retrained


model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)   
model.printLayersInfo()
model.initSession()
results = model.evaluate()

Initializing tokenizer from "/data/SQuAD/vocab.txt" ... Done. (Vocab Size: 30522)

Reading from "Models/BertSquadRR.fbm" ... Done.
Creating the fireball model "Bert-SQuAD" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_EMB           ≤512 2        LR512                    ≤512 768      None                      16,415,232 
S1_L1_LN         ≤512 768                               ≤512 768      None     DO:0.1           1,536      
S2_L1_BERT       ≤512 768      768/3072, 12 heads       ≤512 768      GELU                      5,097,216  
S2_L2_BERT       ≤512 768      768/3072, 12 heads       ≤512 768      GELU                      5,177,088  
S2_L3_BERT       ≤512 768      768/3072, 12 heads       ≤512 768      GELU                      5,207,808  
S2_L4_BERT       ≤512 768      768/3072, 12 heads       ≤512 

## Pruning the model
Here we prune the model using the ``pruneModel`` class method of the model.

In [2]:
prunedFileName = orgFileName.replace('.fbm', 'P.fbm')  # Append 'P' to the filename for "Pruned"
pResults = Model.pruneModel(orgFileName, prunedFileName, mseUb=.00005)


Reading model parameters from "Models/BertSquadRR.fbm" ... Done.
Pruning 272 tensors using 76 workers ... 
   Pruning Parameters:
        mseUb ................ 0.000050
Pruning process complete (11.66 Sec.)
Now saving to "Models/BertSquadRRP.fbm" ... Done.

Number of parameters: 81,965,570 -> 41,818,535 (40,147,035 pruned)
Model File Size: 327,878,772 -> 177,526,445 bytes


## Evaluate the pruned model
Compare the new number of parameters with the original. Let's see the impact of this reduction to the performance of the model.

In [3]:
model = Model.makeFromFile(prunedFileName, testDs=testDs, gpus=gpus)   
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/BertSquadRRP.fbm" ... Done.
Creating the fireball model "Bert-SQuAD" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_EMB           ≤512 2        LR512                    ≤512 768      None                      4,743,508  
S1_L1_LN         ≤512 768                               ≤512 768      None     DO:0.1           1,536      
S2_L1_BERT       ≤512 768      768/3072, 12 heads       ≤512 768      GELU                      2,839,343  
S2_L2_BERT       ≤512 768      768/3072, 12 heads       ≤512 768      GELU                      2,898,736  
S2_L3_BERT       ≤512 768      768/3072, 12 heads       ≤512 768      GELU                      2,909,600  
S2_L4_BERT       ≤512 768      768/3072, 12 heads       ≤512 768      GELU                      3,037,894  
S2_L5_BERT       ≤512 768      768

## Re-training after pruning
Here we make a new model for re-training from the file created above. We then call the ```train``` method of the model to start the re-training.

After re-training, we run the ```evaluate``` function again to see how the re-training improved the performance
of the model.

The re-trained model is then saved to a file appending an 'R' letter (for Re-trained) to the end of the pruned model file name.

In [4]:
trainDs = SquadDSet.makeDatasets("Train", batchSize=128, version=1 )

model = Model.makeFromFile(prunedFileName, trainDs=trainDs, testDs=testDs,
                           batchSize=32, numEpochs=2,
                           regFactor=0.0001,
                           learningRate=(2e-5,4e-6), optimizer='Adam',
                           saveBest=False,
                           gpus=gpus)
model.printNetConfig()
model.initSession()
model.train()
results = model.evaluate()

retrainedFileName = prunedFileName.replace('.fbm', 'R.fbm')  # Append 'R' to the filename for "Retrained"
model.save(retrainedFileName)

Initializing tokenizer from "/data/SQuAD/vocab.txt" ... Done. (Vocab Size: 30522)

Reading from "Models/BertSquadRRP.fbm" ... Done.
Creating the fireball model "Bert-SQuAD" ... Done.

Network configuration:
  Input:                     A tuple of TokenIds and TokenTypes.
  Output:                    2 logit vectors (with length ≤ 512) for start and end indexes of the answer.
  Network Layers:            16
  Tower Devices:             GPU0, GPU1, GPU2, GPU3
  Total Network Parameters:  41,818,535
  Total Parameter Tensors:   272
  Trainable Tensors:         272
  Training Samples:          87,844
  Test Samples:              10,833
  Num Epochs:                2
  Batch Size:                32
  L2 Reg. Factor:            0.0001
  Global Drop Rate:          0   
  Learning Rate: (Exponential Decay)
    Initial Value:           0.00002      
    Final Value:             0.000004     
  Optimizer:                 Adam

+--------+---------+---------------+-----------+-------------------+


## Where do I go from here?

[Quantizing BERT/SQuAD Model](BertSquad-Quantize.ipynb)

[Exporting BERT/SQuAD Model to ONNX](BertSquad-ONNX.ipynb)

[Exporting BERT/SQuAD Model to TensorFlow](BertSquad-TF.ipynb)

[Exporting BERT/SQuAD Model to CoreML](BertSquad-CoreML.ipynb)

________________

[Fireball Playgrounds](../Contents.ipynb)

[Question Answering (BERT/SQuAD)](BertSquad.ipynb)

[Reducing number of parameters of BERT/SQuAD Model](BertSquad-Reduce.ipynb)
